<a href="https://colab.research.google.com/github/YuamLu/Find_NRICM101/blob/main/%E3%80%8CUntitled3_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets hurst statsmodels

In [ ]:
from datasets import load_dataset

datasets = load_dataset('yuan1729/voice_dataset')

datasets

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'ID', 'Sex', 'Age', 'Disease category', 'Narrow pitch range', 'Decreased volume', 'Fatigue', 'Dryness', 'Lumping', 'heartburn', 'Choking', 'Eye dryness', 'PND', 'Smoking', 'PPD', 'Drinking', 'frequency', 'Diurnal pattern', 'Onset of dysphonia ', 'Noise at work', 'Occupational vocal demand', 'Diabetes', 'Hypertension', 'CAD', 'Head and Neck Cancer', 'Head injury', 'CVA', 'Voice handicap index - 10'],
        num_rows: 900
    })
    test: Dataset({
        features: ['audio', 'ID', 'Sex', 'Age', 'Disease category', 'Narrow pitch range', 'Decreased volume', 'Fatigue', 'Dryness', 'Lumping', 'heartburn', 'Choking', 'Eye dryness', 'PND', 'Smoking', 'PPD', 'Drinking', 'frequency', 'Diurnal pattern', 'Onset of dysphonia ', 'Noise at work', 'Occupational vocal demand', 'Diabetes', 'Hypertension', 'CAD', 'Head and Neck Cancer', 'Head injury', 'CVA', 'Voice handicap index - 10'],
        num_rows: 100
    })
})

In [ ]:
from hurst import compute_Hc, random_walk
import numpy as np

# Calculate the Hurst Exponent for the series
def hurst_exp(series):
    H, c, data = compute_Hc(series, kind='random_walk', simplified=True)
    H = np.array([H])
    return H




import statsmodels 

# Calculate the AR parameters for the series
def ar_parameters(series):
    AR = statsmodels.regression.linear_model.burg(series, 14)[0]
    return AR

In [ ]:
def to_lambda(original_list):

  # 构造lambda表达式的字符串模板
  lambda_template = "lambda x: {}"

  # 将每个元组转换为相应的表达式字符串，并将它们拼接起来
  expr_list = ["x['{}'] == {}".format(k, v) for k, v in original_list]
  expr_str = " and ".join(expr_list)

  # 将表达式字符串插入到模板中，并使用eval()函数将其转换为lambda表达式
  lambda_str = lambda_template.format(expr_str)
  lambda_func = eval(lambda_str)

  # 输出lambda表达式
  return lambda_func

In [ ]:
import itertools
from itertools import product

tags = ['Sex','Noise at work']
column_vals = []

for tag in tags:

  types = list(set([i[tag] for i in datasets['train']]))
  column_vals.append([tag,types])
vals_ = ([list(product(*[[i[0]],i[1]])) for i in column_vals])
paras = list(product(*vals_))

ds = []

for i in (list(product(*vals_))):
  ds.append({'tags': i, 'dataset': datasets['train'].filter(to_lambda(i))})

In [ ]:
for d in ds:

  tag = d['tags']
  dataset = d['dataset']

  simple_data = [{'id': i['ID'], 'audio': i["audio"]['array'], 'disease': i['Disease category']} for i in dataset]
  from tqdm import tqdm

  for i in tqdm(range(len(simple_data))):
      simple_data[i]['H_and_AR'] = np.hstack([hurst_exp(simple_data[i]['audio']), ar_parameters(simple_data[i]['audio'])])

  # use sklearn to train a svm classifier model
  # feature is 'H_and_AR' in simple_data, label is 'disease' in simple_data.
  # data split: 90% for training, 10% for testing

  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

  X = np.array([i['H_and_AR'] for i in simple_data])
  y = np.array([i['disease'] for i in simple_data])

  # split data
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)

  from sklearn import svm, tree, neighbors
  from sklearn.ensemble import GradientBoostingClassifier
  from sklearn.linear_model import LogisticRegression
  from sklearn.model_selection import KFold
  from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

  # create a list of models to test
  models = [
      ('SVM', svm.SVC()),
      ('Decision Tree', tree.DecisionTreeClassifier()),
      ('k-NN', neighbors.KNeighborsClassifier()),
      ('Gradient Boosting', GradientBoostingClassifier()),
      ('Logistic Regression', LogisticRegression())
  ]

  print(tag)
  print()
  print()

  # iterate over models
  for name, model in models:
      print(name)
      # perform 10-fold cross-validation
      kf = KFold(n_splits=10, shuffle=True, random_state=3)
      acc_scores = []
      recall_scores = []
      precision_scores = []
      f1_scores = []
      for train_index, test_index in kf.split(X):
          X_train, X_test = X[train_index], X[test_index]
          y_train, y_test = y[train_index], y[test_index]
          # train model
          model.fit(X_train, y_train)
          # test model
          y_pred = model.predict(X_test)
          acc_scores.append(accuracy_score(y_test, y_pred))
          recall_scores.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
          precision_scores.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
          f1_scores.append(f1_score(y_test, y_pred, average='macro', zero_division=0))
      # print average performance metrics over all folds
      print('Accuracy:', round(sum(acc_scores)/len(acc_scores), 4))
      print('Recall:', round(sum(recall_scores)/len(recall_scores), 4))
      print('Precision:', round(sum(precision_scores)/len(precision_scores), 4))
      print('F1:', round(sum(f1_scores)/len(f1_scores), 4))
      print()

  0%|          | 0/207 [00:04<?, ?it/s]


AttributeError: ignored

In [ ]:
!pip install  -q statsmodels